In [ ]:
%matplotlib inline
import pathlib as pl
import numpy as np
import sys
import xugrid
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

import flopy
from flopy.export.shapefile_utils import recarray2shp
from flopy.utils.geometry import Polygon
import flopy.plot.styles as styles

In [ ]:
to_crs_projection = "EPSG:32618"

In [ ]:
ws = pl.Path("../greenport_chd/base/")

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=ws, use_pandas=False)

In [ ]:
sim.model_names

In [ ]:
gwf = sim.get_model("gwf")

In [ ]:
gwf.modelgrid.crs = "EPSG:4456"

In [ ]:
gwf.package_names

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf, layer=0)
mm.plot_bc("GHB")
mm.plot_bc("CHD_SURFACE")

# Create the shapefile of the chd surface data

In [ ]:
fpth = pl.Path("../gis/greenport_chd_chd_surface_4456.shp")

In [ ]:
chd_surface = gwf.get_package("chd_surface")

In [ ]:
chd_spd = pd.DataFrame(chd_surface.stress_period_data.get_data(0))
chd_spd

In [ ]:
vertices = []
for k, i, j in chd_spd["cellid"]:
    vertices.append(gwf.modelgrid.get_cell_vertices(i, j))
polygons = [Polygon(vrt) for vrt in vertices]
len(polygons)

In [ ]:
layer = [k for k, i, j in chd_spd["cellid"]]
row = [i for k, i, j in chd_spd["cellid"]]
column = [j for k, i, j in chd_spd["cellid"]]

In [ ]:
chd_spd["layer"] = layer
chd_spd["row"] = row
chd_spd["column"] = column
chd_spd.drop(columns=["cellid"], inplace=True)

In [ ]:
chd_spd["chd_no"] = chd_spd.index
chd_spd

In [ ]:
recarray2shp(chd_spd.to_records(index=False), geoms=polygons, shpname=fpth, crs=gwf.modelgrid.crs)

In [ ]:
gdf = gpd.read_file(fpth)
gdf

In [ ]:
gdf.crs = gwf.modelgrid.crs

In [ ]:
gdf.plot()

## Reproject the chd surface data shapefile to UTM zone 18N

In [ ]:
fpth = pl.Path("../gis/greenport_chd_chd_surface_utm18n.shp")

In [ ]:
gdf.to_crs(to_crs_projection).to_file(fpth)

In [ ]:
gpd.read_file(fpth).plot()

# Create the shapefile of the ghb data

In [ ]:
fpth = pl.Path("../gis/greenport_chd_ghb_4456.shp")

In [ ]:
ghb = gwf.get_package("ghb")

In [ ]:
ghb_spd = pd.DataFrame(ghb.stress_period_data.get_data(0))
ghb_spd

In [ ]:
ghb_spd["boundname"] = "wetland"
ghb_spd.rename(columns={"bhead": "head"}, errors="raise", inplace=True)
ghb_spd

In [ ]:
vertices = []
for k, i, j in ghb_spd["cellid"]:
    vertices.append(gwf.modelgrid.get_cell_vertices(i, j))
polygons = [Polygon(vrt) for vrt in vertices]
len(polygons)

In [ ]:
layer = [k for k, i, j in ghb_spd["cellid"]]
row = [i for k, i, j in ghb_spd["cellid"]]
column = [j for k, i, j in ghb_spd["cellid"]]

In [ ]:
ghb_spd["layer"] = layer
ghb_spd["row"] = row
ghb_spd["column"] = column
ghb_spd.drop(columns=["cellid"], inplace=True)

In [ ]:
ghb_spd["ghb_no"] = ghb_spd.index

In [ ]:
recarray2shp(ghb_spd.to_records(index=False), geoms=polygons, shpname=fpth, crs=gwf.modelgrid.crs)

In [ ]:
gdf = gpd.read_file(fpth)
gdf

In [ ]:
gdf.crs = gwf.modelgrid.crs

In [ ]:
gdf.plot()

## Reproject the ghb data shapefile to UTM zone 18N

In [ ]:
fpth = pl.Path("../gis/greenport_chd_ghb_utm18n.shp")

In [ ]:
gdf.to_crs(to_crs_projection).to_file(fpth)

In [ ]:
gpd.read_file(fpth).plot()

# Concatenate the two dataframes to create a shapefile for plotting

In [ ]:
fpth = pl.Path("../gis/greenport_chd_plot_4456.shp")

In [ ]:
spd = pd.concat([chd_spd, ghb_spd])

In [ ]:
spd = spd.assign(bnd_no=spd.chd_no.mask(spd.chd_no.isnull(), spd.ghb_no))

In [ ]:
spd["bnd_no"] = spd["bnd_no"].astype(int)

In [ ]:
spd

In [ ]:
vertices = []
for i, j in zip(spd["row"], spd["column"]):
    vertices.append(gwf.modelgrid.get_cell_vertices(i, j))
polygons = [Polygon(vrt) for vrt in vertices]
len(polygons)

In [ ]:
recarray2shp(spd.to_records(index=False), geoms=polygons, shpname=fpth, crs=gwf.modelgrid.crs)

In [ ]:
gdf = gpd.read_file(fpth)
gdf.crs = gwf.modelgrid.crs
gdf

In [ ]:
gdf.plot(column="boundname")

## Reproject the chd and ghb data shapefile to UTM zone 18N

In [ ]:
fpth = pl.Path("../gis/greenport_chd_plot_utm18n.shp")

In [ ]:
gdf.to_crs(to_crs_projection).to_file(fpth)

In [ ]:
gpd.read_file(fpth).plot(column="boundname")